<a href="https://colab.research.google.com/github/WoradeeKongthong/raining_tomorrow_classification/blob/master/08_Raining_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Feature Engineering in part 02  
Outliers : I'll cap the outliers in X_train. And cap the outliers in X_test using the boundaries of X_train.  
Missing values : I'll impute the missing values in categorical features with 'most frequent' value,  
and impute the missing values in numerical features with median.

In [0]:
# libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# **Data Set**

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/raining_tomorrow_classification/master/weatherAUS.csv')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [0]:
# drop RISK_MM column (Recommendation from data description in Kaggle)
df.drop(['RISK_MM'], axis = 1, inplace = True)

# Extract Year, Month, Day from Date column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# drop Date column
df.drop(['Date'], axis = 1, inplace = True)

# select year 2015-2017 to train the model
df = df[df['Year'] > 2014]

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43205 entries, 2109 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       43205 non-null  object 
 1   MinTemp        43002 non-null  float64
 2   MaxTemp        43081 non-null  float64
 3   Rainfall       42799 non-null  float64
 4   Evaporation    19472 non-null  float64
 5   Sunshine       16085 non-null  float64
 6   WindGustDir    40818 non-null  object 
 7   WindGustSpeed  40837 non-null  float64
 8   WindDir9am     40515 non-null  object 
 9   WindDir3pm     41457 non-null  object 
 10  WindSpeed9am   42990 non-null  float64
 11  WindSpeed3pm   41715 non-null  float64
 12  Humidity9am    42696 non-null  float64
 13  Humidity3pm    40781 non-null  float64
 14  Pressure9am    38536 non-null  float64
 15  Pressure3pm    38533 non-null  float64
 16  Cloud9am       25312 non-null  float64
 17  Cloud3pm       22877 non-null  float64
 18  Te

In [0]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

# **Handle the Outliers**
Training Set
- cap the outliers in X_train

Test Set
- cap the outliers in X_test using the upper_cap and lower_cap of X_train

## **Cap the outlier in X_train**

In [0]:
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1

lower_cap = Q1 - 1.5*IQR
upper_cap = Q3 + 1.5*IQR

features = lower_cap.index.values

for feature in features :
  X_train.loc[:,feature] = np.where(X_train.loc[:,feature]<lower_cap[feature],lower_cap[feature], X_train.loc[:,feature])
  X_train.loc[:,feature] = np.where(X_train.loc[:,feature]>upper_cap[feature],upper_cap[feature], X_train.loc[:,feature])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## **Cap the outlier in X_test**

In [0]:
for feature in features :
  X_test.loc[:,feature] = np.where(X_test.loc[:,feature]<lower_cap[feature],lower_cap[feature], X_test.loc[:,feature])
  X_test.loc[:,feature] = np.where(X_test.loc[:,feature]>upper_cap[feature],upper_cap[feature], X_test.loc[:,feature])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# **Random Forest**

**Create Preprocessor : ColumnTransformer of numerical and categorical features**

In [0]:
numerical_features = [x for x in X.columns if df[x].dtype != 'object']

numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', MinMaxScaler())
])

categorical_features = [x for x in X.columns if df[x].dtype == 'object']

categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numerical_features),
          ('cat', categorical_transformer, categorical_features)
    ]
)

**Create model**

In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')

**Create Pipeline**

In [0]:
clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
])

# **Model Evaluation**

## **Cross Validation**

In [0]:
accuracy = cross_val_score(clf,X_train,y_train,cv=10)
print('accuracy : ', accuracy)
print('mean : ', accuracy.mean())
print('std : ', accuracy.std())

accuracy :  [0.84119178 0.83714203 0.83540642 0.84061325 0.84751157 0.83072917
 0.83072917 0.83738426 0.83969907 0.8365162 ]
mean :  0.8376922930125671
std :  0.004745526295881819


Note : low bias and low variance

## **Training and Test evaluation**

In [0]:
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n\nTraining and Test Sets result')
print('\naccuracy score : ', accuracy_score(y_test,y_pred))
print('\nconfusion matrix : \n', confusion_matrix(y_test, y_pred))
print('\nclassification report : \n', classification_report(y_test,y_pred))

print('Training set score : ',clf.score(X_train,y_train))
print('Test set score : ',clf.score(X_test,y_test))



Training and Test Sets result

accuracy score :  0.8335840759171392

confusion matrix : 
 [[6382  326]
 [1112  821]]

classification report : 
               precision    recall  f1-score   support

          No       0.85      0.95      0.90      6708
         Yes       0.72      0.42      0.53      1933

    accuracy                           0.83      8641
   macro avg       0.78      0.69      0.72      8641
weighted avg       0.82      0.83      0.82      8641

Training set score :  0.9865177641476681
Test set score :  0.8335840759171392


Note : slightly overfitting

# **Improve the model**

## add max_depth

In [0]:
for i in [5,6,7,8,9,10]:
  model = RandomForestClassifier(n_estimators = 10, max_depth = i, criterion = 'entropy')
  clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
  ])

  clf.fit(X_train,y_train)

  print('Random Forest : max_depth = {}'.format(i))
  print('Training set score : ',clf.score(X_train,y_train))
  print('Test set score : ',clf.score(X_test,y_test))

Random Forest : max_depth = 5
Training set score :  0.8189445666010878
Test set score :  0.8093970605254022
Random Forest : max_depth = 6
Training set score :  0.8310380742969564
Test set score :  0.8234000694364079
Random Forest : max_depth = 7
Training set score :  0.8331790302048374
Test set score :  0.8212012498553408
Random Forest : max_depth = 8
Training set score :  0.8387050109940979
Test set score :  0.8266404351348223
Random Forest : max_depth = 9
Training set score :  0.8464008795278324
Test set score :  0.825135979631987
Random Forest : max_depth = 10
Training set score :  0.8536338386760791
Test set score :  0.8306908922578405


Note : max_depth helps reduce overfitting

## GridSearch

In [0]:
model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'model__max_depth':[8],
    'model__n_estimators':[10,20,30,40,50],
}

search = GridSearchCV(clf, param_grid, n_jobs=-1)
search.fit(X_train, y_train)

print("Best parameter : ", search.best_params_)
print("Best score : ", search.best_score_)

Best parameter :  {'model__max_depth': 8, 'model__n_estimators': 50}
Best score :  0.831964054070163


In [0]:
# train-test on the best parameter

model = RandomForestClassifier(n_estimators = 50,max_depth = 8, criterion='entropy')

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n\nTraining and Test Sets result')
print('\naccuracy score : ', accuracy_score(y_test,y_pred))
print('\nconfusion matrix : \n', confusion_matrix(y_test, y_pred))
print('\nclassification report : \n', classification_report(y_test,y_pred))

print('Training set score : ',clf.score(X_train,y_train))
print('Test set score : ',clf.score(X_test,y_test))



Training and Test Sets result

accuracy score :  0.829649346140493

confusion matrix : 
 [[6531  177]
 [1295  638]]

classification report : 
               precision    recall  f1-score   support

          No       0.83      0.97      0.90      6708
         Yes       0.78      0.33      0.46      1933

    accuracy                           0.83      8641
   macro avg       0.81      0.65      0.68      8641
weighted avg       0.82      0.83      0.80      8641

Training set score :  0.8432183775026039
Test set score :  0.829649346140493


Note : Overfitting is reduced